<a href="https://colab.research.google.com/github/NimrodDev/TensorFlow/blob/main/NLP%20with%20RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install keras

In [10]:
import keras

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from keras.datasets import imdb
from keras.preprocessing import sequence
import numpy as np
import os
VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
#Lets look at one review
train_data[0]

In [3]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation= "sigmoid")
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
model.compile(loss= "binary_crossentropy", optimizer= "rmsprop", metrics=['acc'])

history = model.fit(train_data, train_labels, epochs = 10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 49s 74ms/step - loss: 0.4463 - acc: 0.7905 - val_loss: 0.3274 - val_acc: 0.8612
Epoch 2/10
625/625 [==============================] - 20s 32ms/step - loss: 0.2602 - acc: 0.8995 - val_loss: 0.3896 - val_acc: 0.8366
Epoch 3/10
625/625 [==============================] - 13s 21ms/step - loss: 0.2024 - acc: 0.9265 - val_loss: 0.3168 - val_acc: 0.8866
Epoch 4/10
625/625 [==============================] - 9s 15ms/step - loss: 0.1638 - acc: 0.9427 - val_loss: 0.3678 - val_acc: 0.8658
Epoch 5/10
625/625 [==============================] - 12s 19ms/step - loss: 0.1361 - acc: 0.9532 - val_loss: 0.3012 - val_acc: 0.8784
Epoch 6/10
625/625 [==============================] - 8s 13ms/step - loss: 0.1167 - acc: 0.9612 - val_loss: 0.3236 - val_acc: 0.8710
Epoch 7/10
625/625 [==============================] - 9s 14ms/step - loss: 0.0953 - acc: 0.9703 - val_loss: 0.3306 - val_acc: 0.8760
Epoch 8/10
625/625 [==============================] - 9s 14ms/ste

In [7]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 5s 6ms/step - loss: 0.5300 - acc: 0.8266
[0.5300352573394775, 0.8265600204467773]


In [11]:
# Here we make predictions
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [ ]:
# While were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] +""

  return text[:-1]

In [12]:
# Now time to make prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was so awesome! I really loved it and would watch it again because it was amazingly great"
predict(positive_review)

negative_review ="That movie sucked, i hated it and wouldn't watch it again. Was one of the worst things i've ever watched"
predict(negative_review)

1/1 [==============================] - 0s 357ms/step
[0.93577486]
1/1 [==============================] - 0s 24ms/step
[0.19414707]
